In [34]:
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt


# Parameters for the Ising Hamiltonian
# L has to be even! Due to K only being able to control even Ls!
L = 4

# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.IsingHamiltonian(field, 1, 0, 0).as_matrix().toarray()
#hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
perms = [[i for i in range(L)], [i for i in range(1, L)]+[0]]

eigenvalues, eigenvectors = LA.eig(hamil)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]
print("Ground State Energy", -eigenvalues_sort[-1].real)

Ground State Energy -4.0


In [35]:
def unitary_approximation(V):
    U, _, Vh = np.linalg.svd(V)
    unitary_approx = U @ Vh
    return unitary_approx

In [36]:
import scipy

ket_0 = np.array([[1],[0]])
ket_1 = np.array([[0],[1]])
rho_0_anc = ket_0 @ ket_0.T
rho_1_anc = ket_1 @ ket_1.T
U = scipy.linalg.expm(-1j*hamil)
U_back = scipy.linalg.expm(1j*hamil)
cU = np.kron(rho_0_anc, U_back) + np.kron(rho_1_anc, U)

In [37]:
I2 = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [38]:
import sys
sys.path.append("../ccU")
from utils import otimes, applyG
from ansatz import ansatz, ansatz_grad_vector
from hessian import ansatz_hessian_matrix

V1 = np.kron(X, I2)
V1 = np.kron(rho_1_anc, np.eye(4)) + np.kron(rho_0_anc, V1)
V2 = np.kron(I2, I2)
V2 = np.kron(rho_1_anc, np.eye(4)) + np.kron(rho_0_anc, V2)
W1 = V1
W2 = V2
Glist = [V1, V2, W1, W2]

np.linalg.norm(cU - ansatz(Glist, U, L, perms), ord=2)

0.0

In [39]:
from optimize import err

err(Glist, U, L, perms, cU)

-32.0

In [40]:
grad = ansatz_grad_vector(Glist, cU, U, L, perms, flatten=True)
np.linalg.norm(grad, ord=2)

# Gradient is indeed 0 at the optimal/exact solution. -> Suggested me that my gradient is correct (?)

1.4060808631937454e-18

In [41]:
# Initial params.
V1 = np.kron(Z, I2)
V1 = np.kron(rho_1_anc, np.eye(4)) + np.kron(rho_0_anc, V1)
V2 = np.kron(I2, I2)
V2 = np.kron(rho_1_anc, np.eye(4)) + np.kron(rho_0_anc, V2)
W1 = V1
W2 = V2
Glist = [V1, V2, W1, W2]

np.linalg.norm(cU - ansatz(Glist, U, L, perms), ord=2)

1.5136049906158566

In [48]:
from utils import antisymm, antisymm_to_real
import numpy as np

epsilon = 1e-5

# Random perturbation in the same structure as Glist
Zlist = [np.random.randn(8,8) + 1j*np.random.randn(8,8) for G in Glist]
# Project each Z matrix onto anti-Hermitian matrices
Zlist = [antisymm(Z) for Z in Zlist]

# Perturb Glist
Glist_plus = [G + epsilon * Z for G, Z in zip(Glist, Zlist)]
Glist_minus = [G - epsilon * Z for G, Z in zip(Glist, Zlist)]

f_plus = err(Glist_plus, U, L, perms, cU)
f_minus = err(Glist_minus, U, L, perms, cU)

# Finite difference estimate
grad_fd = (f_plus - f_minus) / (2 * epsilon)

# Compute analytical gradient
grad = -ansatz_grad_vector(Glist, cU, U, L, perms, flatten=True)  # This gives a real flat vector

# Flatten Zlist in the same way as grad
Zlist_flat = np.concatenate([antisymm_to_real(Z) for Z in Zlist]).ravel()

# Inner product
inner_product = np.dot(grad, Zlist_flat)

print("Finite difference gradient:", grad_fd)
print("Inner product with analytical gradient:", inner_product)

# Should be close:
print("Relative error:", np.abs(grad_fd - inner_product) / np.abs(grad_fd))

# !! Gradient does not seem to be quite correct...

Finite difference gradient: 13.32201637520569
Inner product with analytical gradient: 11.065253180583905
Relative error: 0.16940102241744473
